In [28]:
import pandas as pd
import numpy as np
import os

In [29]:
perspective_files = os.listdir('data/perspective/')
politeness_files = os.listdir('data/politeness/')
sentiment_files = os.listdir('data/sentiment/')

all_files = set(perspective_files + politeness_files)
all_files

{'CatGifs.tsv',
 'CatsStandingUp.tsv',
 'Dogtraining.tsv',
 'FULLCOMMUNISM.tsv',
 'GreenParty.tsv',
 'Liberal.tsv',
 'Libertarian_short.tsv',
 'NeutralPolitics.tsv',
 'OSU.tsv',
 'Republican.tsv',
 'StartledCats.tsv',
 'TrollXChromosomes_short.tsv',
 'TwoXChromosomes_short.tsv',
 'cats.tsv',
 'communism.tsv',
 'democrats.tsv',
 'demsocialist.tsv',
 'dogpictures.tsv',
 'dogs_short.tsv',
 'msu.tsv',
 'progressive.tsv',
 'puppies.tsv',
 'republicans.tsv',
 'socialism.tsv',
 'socialism_short.tsv',
 'uofm.tsv'}

In [30]:
to_merge = {}

for f in all_files:
    if f.endswith('.tsv'):
        tmp = []
        if f in perspective_files:
            tmp.append('data/perspective/')
        if f in politeness_files:
            tmp.append('data/politeness/')
        if f in sentiment_files:
            tmp.append('data/sentiment/')
        to_merge[f] = tmp
to_merge

{'CatGifs.tsv': ['data/perspective/', 'data/politeness/', 'data/sentiment/'],
 'CatsStandingUp.tsv': ['data/perspective/', 'data/politeness/'],
 'Dogtraining.tsv': ['data/perspective/'],
 'FULLCOMMUNISM.tsv': ['data/perspective/'],
 'GreenParty.tsv': ['data/perspective/',
  'data/politeness/',
  'data/sentiment/'],
 'Liberal.tsv': ['data/perspective/', 'data/politeness/'],
 'Libertarian_short.tsv': ['data/perspective/', 'data/politeness/'],
 'NeutralPolitics.tsv': ['data/perspective/'],
 'OSU.tsv': ['data/perspective/', 'data/politeness/'],
 'Republican.tsv': ['data/perspective/'],
 'StartledCats.tsv': ['data/perspective/',
  'data/politeness/',
  'data/sentiment/'],
 'TrollXChromosomes_short.tsv': ['data/perspective/', 'data/politeness/'],
 'TwoXChromosomes_short.tsv': ['data/perspective/', 'data/politeness/'],
 'cats.tsv': ['data/perspective/', 'data/politeness/', 'data/sentiment/'],
 'communism.tsv': ['data/perspective/', 'data/politeness/'],
 'democrats.tsv': ['data/perspective/', 

In [31]:
tmp = pd.read_csv('data/politeness/CatGifs.tsv', sep='\t')
tmp.columns.values

array(['approved_at_utc', 'approved_by', 'archived', 'author',
       'author_cakeday', 'author_flair_css_class', 'author_flair_text',
       'banned_at_utc', 'banned_by', 'body', 'can_gild', 'can_mod_post',
       'collapsed', 'collapsed_reason', 'controversiality', 'created',
       'created_utc', 'distinguished', 'downs', 'edited', 'gilded', 'id',
       'is_submitter', 'likes', 'link_id', 'name', 'num_reports',
       'parent_id', 'removal_reason', 'replies', 'report_reasons',
       'retrieved_on', 'saved', 'score', 'score_hidden', 'stickied',
       'subreddit', 'subreddit_id', 'ups', 'politeness'], dtype=object)

In [32]:
for k,v in to_merge.items():
    print(k)
    df = pd.DataFrame()
    for p in v:
        tmp = pd.read_csv(p+k, sep='\t')
        if df.shape[1]==0:
            df=tmp
        else:
            keep = list(set(tmp.columns.values) - set(df.columns.values))
            keep.append('id')
            tmp = tmp[keep]
            df = df.merge(tmp, on='id')
    df.to_csv('data/merged/'+k, sep='\t', index=False)
    

demsocialist.tsv
puppies.tsv
progressive.tsv


/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,7,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,6,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


dogpictures.tsv


/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Liberal.tsv


...

StartledCats.tsv
dogs_short.tsv
TwoXChromosomes_short.tsv


/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,5,6,7,11,12,13,14,23,26,33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,4,5,6,10,11,12,13,22,25,32,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


GreenParty.tsv
Republican.tsv


/home/jwlock/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,6,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


socialism_short.tsv


In [33]:
print('done!')

done!
